In [ ]:
%pip install roboflow
%pip install ultralytics
%pip install python-dotenv

In [ ]:
import torch
import ultralytics
import sys
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("API_KEY")

print("Python:", sys.version)
print("Torch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("Ultralytics:", ultralytics.__version__)

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("vsss").project("vsss-inimodel-y1u7t")
version = project.version(5)
dataset = version.download("yolov8")      

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 batch=4 plots=True

In [ ]:
Image(filename='/home/dany/vsss_model/runs/detect/train4/results.png', width=600)

In [ ]:
from IPython.display import Image
Image(filename='/home/dany/vsss_model/runs/detect/train4/val_batch2_pred.jpg', width=600)

In [ ]:
from ultralytics import YOLO
from IPython.display import Image
import glob

model = YOLO('/home/dany/vsss_model/runs/detect/train4/weights/best.pt')
results = model.predict('/home/dany/vsss_model/vsss-iniModel-5/test/images/captura_23_png.rf.a30df663d57d2f3ae14f6aab42e5b28d.jpg', save=True)
        


# La imagen predicha se guarda en runs/detect/predict/
Image(filename='runs/detect/predict/captura_23_png.rf.a30df663d57d2f3ae14f6aab42e5b28d.jpg', width=600)

In [ ]:
from ultralytics import YOLO
from IPython.display import Image, display
import glob
from PIL import Image as PILImage

model = YOLO('/home/dany/vsss_model/runs/detect/train4/weights/best.pt')

# Carpeta con imágenes de prueba
test_images_folder = '/home/dany/vsss_model/vsss-iniModel-5/test/images/'
image_paths = glob.glob(test_images_folder + '*.jpg')

# Margen negativo para recortar más ajustado (en píxeles)
margin = -10

for img_path in image_paths:
    results = model.predict(img_path)
    im = PILImage.open(img_path)
    for box in results[0].boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = box
        # Aplica margen negativo para recortar más ajustado
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(im.width, x2 + margin)
        y2 = min(im.height, y2 + margin)
        crop = im.crop((x1, y1, x2, y2))
        display(crop)